# Basic dynamic model

In [4]:
DATA_PATH = '../backend/data/dynamic_dataset'
DATA_PATH = '../backend/dynamic_signs/frames'
CSV_PATH = '../out.csv'

In [5]:
#Load DATA
from sign.training.landmark_extraction.MediaPiper import MediaPiper

mp = MediaPiper()
gestures = mp.process_dynamic_gestures_from_folder(DATA_PATH)

In [6]:
[(gesture.label, f"Length: {len(gesture.results)}") for gesture in gestures]

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


[]

In [7]:
import numpy as np
from sign.trajectory import TrajectoryBuilder, trajectory

bob = TrajectoryBuilder()

gesture_trajector_map: dict[str, list[trajectory]] = {}

for gesture in gestures:
    gesture_trajector_map[gesture.label] = []
    for sequence in gesture.results:
        new_sequence = []
        for image_mp_res in sequence:
            hand_landmarks = np.array(image_mp_res.multi_hand_landmarks)
            new_sequence.append(hand_landmarks)
        new_sequence = bob.extract_keyframes_sample(new_sequence)
        sequence_as_np_array = np.array(new_sequence)
        sequence_trajectory = bob.make_trajectory(sequence_as_np_array)

        gesture_trajector_map[gesture.label].append(sequence_trajectory)



🔥🔥 TrajectoryBuilder is now running in BERTRAM_MODE 🔥🔥


In [8]:
X = []
y = []

for k,gesture in gesture_trajector_map.items():
    min = len(gesture[0].directions)
    for trajectory in gesture[1:]:
        length = len(trajectory.directions)
        if min > length:
            min = length
    for trajectory in gesture:
        if len(trajectory.directions) > min:
            for i in range(len(trajectory.directions) - min):
                trajectory.directions.pop()


    for trajector in gesture:
        xyz  = []
        for te in trajector.directions:
            xyz.append(te.x.value)
            xyz.append(te.y.value)
            xyz.append(te.z.value)
        X.append(np.array(xyz))
        y.append(k)

X = np.array(X)
y = np.array(y)
X

array([], dtype=float64)

In [9]:
from sklearn.linear_model import LinearRegression, LogisticRegression

model = LogisticRegression()
model.fit(X,y)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0xffff963547d0>>
Traceback (most recent call last):
  File "/mnt/repositories/Bsc-Sign-Language-Recon/backend/.venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
if False:
    from joblib import dump
    dump(model, 'dynamic_model.joblib')

### Basic Gesture Recognizer but with landmarks :eyes:

So the idea is that we will still use the "basic" trajectory, all landmarks reduced to mean x,y,z values.
But, next to this trajectory we will attach preprocessed landmarks for each of the frames.
- That is, we will only attach preprocessed landmarks corresponding to the chosen key frames.
- The preprocessing step is EXACTLY the same as for static images.
  - Which is also why the 'Z' values have been cut in the following steps. No other reason than our methods already in use for preprocssing do so.

In [2]:
from math import e
from typing import Tuple
import random
import numpy as np
from sign.training.landmark_extraction.MediaPiper import DynamicGesture
from sign.trajectory import TrajectoryBuilder, trajectory
from sign.landmarks import NormalizedLandmark, pre_process_landmark, calc_landmark_list
from dynamic_signs.csv_reader import csv_reader
bob = TrajectoryBuilder(target_len=24)
def training_trajectories_with_normalized_landmarks(gestures:list[DynamicGesture]) -> dict[str, list[np.ndarray]]:

    gesture_trajector_map: dict[str, list[np.ndarray]] = {}

    for gesture in gestures:
        gesture_trajector_map[gesture.label] = []
        for sequence in gesture.results:
            new_sequence : list[Tuple] = []
            #flatmarks: list[np.ndarray] = []
            for image_mp_res in sequence:
                if image_mp_res.multi_hand_landmarks is not None:
                    hand_landmarks = np.array(image_mp_res.multi_hand_landmarks)

                    normalized_landmarks_for_image = pre_process_landmark(
                        calc_landmark_list(image_mp_res.multi_hand_landmarks))
                    flatmark = np.array(normalized_landmarks_for_image)

                    new_sequence.append((hand_landmarks, flatmark))

            #new_sequence = bob.extract_keyframes_sample(new_sequence)
            random.seed(42)
            res = [new_sequence[0]]
            res.extend(random.sample(new_sequence[1:-1], bob.target_len - 2))
            res.append(new_sequence[-1])
            
            
            hand_landmark_seq = [ hand_landmark for hand_landmark, _ in res ]
            flatmarks = [ flatmark for _, flatmark in res ] #without Z values
            sequence_as_np_array = np.array(hand_landmark_seq)
            sequence_trajectory = bob.make_trajectory(sequence_as_np_array)

            xyz = []
            for trajectory_elm in sequence_trajectory.directions:
                xyz.append(trajectory_elm.x.value)
                xyz.append(trajectory_elm.y.value)
                xyz.append(trajectory_elm.z.value)
            xyz = np.array(xyz)

            
            for flat_landmark in flatmarks:
                xyz = np.append(xyz, flat_landmark)

            gesture_trajector_map[gesture.label].append(xyz)

    return gesture_trajector_map


def extract_training_data_and_labels_from_dynamic_gesture_map(gesture_map: dict[str, list[np.ndarray]]) -> Tuple[list[np.ndarray], list[str]]:
    trajectories_and_landmarks: list[np.ndarray] = []
    labels : list[str] = []
    for label, label_data in gesture_map.items():
        for data in label_data:
            labels.append(label)
            trajectories_and_landmarks.append(data)

    return trajectories_and_landmarks, labels

reader = csv_reader()



def prune_training_data_and_labels_from_dynamic_gesture_csv(input: dict[str, dict[int, list[float]]]) -> dict[str, list[np.ndarray]]:
    target_length = 24*3*21
    bob = TrajectoryBuilder(target_len=target_length)
    res: dict[str, list[np.ndarray]] = {}
    for label, videos in input.items():
        for _, frames in videos.items():
            existing = res.get(label)
            if len(frames) < target_length:
                frames = bob.pad_sequences_of_landmarks(frames)
            else: 
                frames = bob.extract_keyframes_sample(frames)
            frames = np.array(frames)
            if existing is not None:
                existing.append(frames)
            else:
                res[label] = [frames]
                
    return res
    
def extract_training_data_and_labels_from_dynamic_gesture_csv() -> Tuple[list[np.ndarray], list[str]]:
    unpruned = reader.extract_landmarks("../backend/out.csv")
    pruned = prune_training_data_and_labels_from_dynamic_gesture_csv(unpruned)
    normalized_landmarks = {}
    
    for key, val in pruned.items():
        for seq in val:
            trajectory = bob.make_trajectory(seq.reshape(-1, 21, 3))
            landmarks = []
            for i in range(0, len(seq), 3):
                landmark = NormalizedLandmark()
                landmark.x = seq[i]
                landmark.y = seq[i+1]
                landmark.z = seq[i+2]
                landmarks.append(landmark)
            normalized_landmarks_for_video = pre_process_landmark(calc_landmark_list(landmarks))
            existing = normalized_landmarks.get(key)
            asd = trajectory.to_float_list()
            asd.extend(normalized_landmarks_for_video)
            with_trajectories = np.array(asd)
            print(with_trajectories)
            if existing is None:
                normalized_landmarks[key] = [with_trajectories]
            else:
                existing.append(with_trajectories)
    return extract_training_data_and_labels_from_dynamic_gesture_map(normalized_landmarks)

extracted_data,extracted_labels = extract_training_data_and_labels_from_dynamic_gesture_csv()
len(extracted_data),len(extracted_labels)
# map = training_trajectories_with_normalized_landmarks(gestures)
# train_data, train_labels  = extract_training_data_and_labels_from_dynamic_gesture_map(map)
# len(extracted_data),len(extracted_labels)


🔥🔥 TrajectoryBuilder is now running in BERTRAM_MODE 🔥🔥
🔥🔥 TrajectoryBuilder is now running in BERTRAM_MODE 🔥🔥
(-1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 0.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 0.0, 1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0)
[-1.          1.          1.         ... -0.24040404 -0.11414141
 -0.26969697]
(-1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 0.0, 1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 0.0, 1.0, 1.0, 1.0, -1.0, 1.0, 0.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 0.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0)
[-1.         -1.          1.         ... -0.0807

(708, 708)

In [3]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression

# model_logistic = LogisticRegression()
# model_logistic.fit(extracted_data, extracted_labels)


model_svm = make_pipeline(StandardScaler(),
                          SVC(kernel="poly", degree=6, coef0=1))
model_svm.fit(extracted_data, extracted_labels)

if True:
    from joblib import dump
    dump(model_svm, 'dynamic_model.joblib')

In [ ]:
def sample_tuple_hand_landmarks_normalized(tuple_list:list[Tuple]):
    random.seed(42)
    res = [tuple_list[0]]
    res.extend(random.sample(tuple_list[1:-1], bob.target_len - 2))
    res.append(tuple_list[-1])
    
    
    hand_landmark_seq = [ hand_landmark for hand_landmark, _ in res ]
    flatmarks = [ flatmark for _, flatmark in res ] #without Z values
    return hand_landmark_seq, flatmarks

def mp_landmarks_to_dynamic_prediction_ready(landmarks):
    tuple_list = []
    for landmark in landmarks:
        pre_processed = pre_process_landmark(calc_landmark_list(landmark))
        hand_landmark = np.array(landmark)
        tuple_list.append((hand_landmark, pre_processed))
    
    key_frame_tuple = sample_tuple_hand_landmarks_normalized(tuple_list)

    flatmarks = key_frame_tuple[1]
    sequence_as_np_array = np.array(key_frame_tuple[0])
    trajectory = bob.make_trajectory(sequence_as_np_array)

    xyz = []
    for trajectory_elm in trajectory.directions:
        xyz.append(trajectory_elm.x.value)
        xyz.append(trajectory_elm.y.value)
        xyz.append(trajectory_elm.z.value)
    xyz = np.array(xyz)

    
    for flat_landmark in flatmarks:
        xyz = np.append(xyz, flat_landmark)

    return xyz
    

In [ ]:
#Test on a single Z
TEST_PATH = '../backend/dynamic_signs/test'

test_data = mp.process_dynamic_gestures_from_folder(TEST_PATH)
#res = list(filter(lambda x : x.label == 'Z', res))
labels_test = [result.label for result in test_data]
labels_test

In [ ]:
prediction_material = []
for dynamic_gesture in test_data:
    for sequence in dynamic_gesture.results:
        hand_landmarks_for_seq = [mp_result.multi_hand_landmarks for mp_result in sequence 
                                  if mp_result.multi_hand_landmarks is not None]
        prediction_material.append(
            mp_landmarks_to_dynamic_prediction_ready(hand_landmarks_for_seq)
        )

print("Logistis predictions: ", model_logistic.predict(prediction_material))
print("Support vector machine: ", model_svm.predict(prediction_material))

##### Metrics for Dynamic Gesture recogniser

In [ ]:
from sklearn.metrics import classification_report
cr = classification_report(model_svm.predict(prediction_material), labels_test)
print(cr)